In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import plotly.express as px
import plotly.graph_objects as go

In [2]:
train = pd.read_csv(r'../input/tabular-playground-series-aug-2022/train.csv')
train.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


In [3]:
test = pd.read_csv(r'../input/tabular-playground-series-aug-2022/test.csv')
test.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,26570,F,119.57,material_5,material_6,6,4,6,9,6,...,18.654,10.802,15.909,18.070,13.772,13.659,16.825,13.742,17.710,634.612
1,26571,F,113.51,material_5,material_6,6,4,11,8,0,...,19.368,12.032,13.998,NaN,12.473,17.468,16.708,14.776,14.102,537.037
2,26572,F,112.16,material_5,material_6,6,4,8,12,4,...,17.774,11.743,17.046,18.086,10.907,13.363,15.737,17.065,16.021,658.995
3,26573,F,112.72,material_5,material_6,6,4,8,11,10,...,18.948,11.790,18.165,16.163,10.933,15.501,15.667,12.620,16.111,594.301
4,26574,F,208.00,material_5,material_6,6,4,14,16,8,...,19.141,12.370,14.578,17.849,11.941,16.070,16.183,13.324,17.150,801.044


In [4]:
sub = pd.read_csv(r'../input/tabular-playground-series-aug-2022/sample_submission.csv')
sub.head()

,id,failure
0,26570,0.0
1,26571,0.0
2,26572,0.0
3,26573,0.0
4,26574,0.0


In [5]:
print(f'train set have {train.shape[0]} rows and {train.shape[1]} columns.')
print(f'test set have {test.shape[0]} rows and {test.shape[1]} columns.') 
print(f'sample_submission set have {sub.shape[0]} rows and {sub.shape[1]} columns.') 

train set have 26570 rows and 26 columns.
test set have 20775 rows and 25 columns.
sample_submission set have 20775 rows and 2 columns.


In [6]:
train.dtypes

id                  int64
product_code       object
loading           float64
attribute_0        object
attribute_1        object
attribute_2         int64
attribute_3         int64
measurement_0       int64
measurement_1       int64
measurement_2       int64
measurement_3     float64
measurement_4     float64
measurement_5     float64
measurement_6     float64
measurement_7     float64
measurement_8     float64
measurement_9     float64
measurement_10    float64
measurement_11    float64
measurement_12    float64
measurement_13    float64
measurement_14    float64
measurement_15    float64
measurement_16    float64
measurement_17    float64
failure             int64
dtype: object

In [7]:
train.isnull().sum()

id                   0
product_code         0
loading            250
attribute_0          0
attribute_1          0
attribute_2          0
attribute_3          0
measurement_0        0
measurement_1        0
measurement_2        0
measurement_3      381
measurement_4      538
measurement_5      676
measurement_6      796
measurement_7      937
measurement_8     1048
measurement_9     1227
measurement_10    1300
measurement_11    1468
measurement_12    1601
measurement_13    1774
measurement_14    1874
measurement_15    2009
measurement_16    2110
measurement_17    2284
failure              0
dtype: int64

In [8]:
train.nunique()

id                26570
product_code          5
loading           11950
attribute_0           2
attribute_1           3
attribute_2           4
attribute_3           4
measurement_0        29
measurement_1        30
measurement_2        25
measurement_3      4721
measurement_4      4692
measurement_5      4671
measurement_6      4704
measurement_7      4734
measurement_8      4713
measurement_9      4708
measurement_10     6177
measurement_11     6526
measurement_12     6392
measurement_13     5271
measurement_14     6389
measurement_15     6577
measurement_16     7035
measurement_17    23612
failure               2
dtype: int64

In [9]:
! pip install --ignore-installed --pre pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.5/544.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / - \ | / done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.8/793.8 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━

In [10]:
from pycaret.classification import setup, compare_models, blend_models, finalize_model, predict_model, get_config

In [11]:
def pycaret_model(train, target,test, n_select, fold,opt):
    print('Setup Your Data....')
    setup(data=train,
          target=target)
  
    print('Comparing Models....')
    best = compare_models(sort = opt,n_select=n_select, fold = fold)
    
    print('Blending Models....')
    blended = blend_models(estimator_list= best, fold=fold, optimize=opt)
#     pred_holdout = predict_model(blended)
    
#     print('Finallizing Models....')
#     final_model = finalize_model(blended)
#     print('Done...!!!')

    prep_pipe = get_config("pipeline")
    prep_pipe.steps.append(['final_model', blended])
    pred = prep_pipe.predict_proba(test)[:,1]
    
    return pred

In [12]:
result = pycaret_model(train,'failure',test, 3, 5, 'AUC')

Setup Your Data....


,Description,Value
0,Session id,7490
1,Target,failure
2,Target type,Binary
3,Original data shape,"(26570, 26)"
4,Transformed data shape,"(26570, 32)"
5,Transformed train set shape,"(18598, 32)"
6,Transformed test set shape,"(7972, 32)"
7,Ordinal features,1
8,Numeric features,22
9,Categorical features,3


Comparing Models....


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7873,0.5923,0.0030,0.4550,0.0060,0.0034,0.0225,3.8300
lda,Linear Discriminant Analysis,0.7873,0.5920,0.0073,0.5013,0.0144,0.0082,0.0387,0.6360
gbc,Gradient Boosting Classifier,0.7861,0.5818,0.0018,0.1425,0.0035,-0.0007,-0.0054,6.9300
nb,Naive Bayes,0.7818,0.5759,0.0405,0.3762,0.0731,0.0332,0.0612,0.6340
ada,Ada Boost Classifier,0.7855,0.5720,0.0053,0.2905,0.0104,0.0022,0.0104,1.9820
catboost,CatBoost Classifier,0.7858,0.5669,0.0043,0.2736,0.0085,0.0017,0.0081,12.8260
lightgbm,Light Gradient Boosting Machine,0.7841,0.5623,0.0058,0.2098,0.0113,0.0000,-0.0002,2.0980
qda,Quadratic Discriminant Analysis,0.7801,0.5620,0.0369,0.3424,0.0666,0.0263,0.0485,0.6020
rf,Random Forest Classifier,0.7868,0.5562,0.0008,0.1667,0.0015,-0.0003,-0.0030,5.3040
et,Extra Trees Classifier,0.7873,0.5540,0.0005,0.3000,0.0010,0.0004,0.0045,2.3820


Processing:   0%|          | 0/71 [00:00<?, ?it/s]

Blending Models....


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7874,0.5947,0.0038,0.5000,0.0075,0.0043,0.0282
1,0.7866,0.5778,0.0013,0.2000,0.0025,-0.0002,-0.0011
2,0.7868,0.6022,0.0038,0.3750,0.0075,0.0033,0.0184
3,0.7876,0.5877,0.0013,0.5000,0.0025,0.0015,0.0163
4,0.7876,0.5953,0.0025,0.6667,0.0050,0.0034,0.0315
Mean,0.7872,0.5915,0.0025,0.4483,0.0050,0.0025,0.0187
Std,0.0004,0.0083,0.0011,0.1550,0.0022,0.0016,0.0114


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [13]:
sub['failure'] = result
sub.to_csv('sub.csv',index=False)

In [14]:
sub.head()

,id,failure
0,26570,0.199319
1,26571,0.168680
2,26572,0.182657
3,26573,0.197452
4,26574,0.351411
